In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install GPUtil
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
#!pip install wandb
import json
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy

import numpy as np
import pandas as pd


import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
import torch

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)

torch.manual_seed(RANDOM_SEED)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:

def ConvertLabel2ModelLabel(label):
  if label == 'Positive':
    o = 2
  elif label == 'Neutral':
    o = 1
  elif label == 'Negative':
    o = 0
  elif label == 'spam':
    o = 1
  else: 
    print(" error at ConvertLabel2ModelLabel label is " , label)
    o = np.nan
  return o


from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

In [4]:
train_path = '/content/drive/MyDrive/fyp/fyp2/final models and datasets/dataset/dataset_new_with_self_label_final_5.0:3.0:3.0/train_dataset.csv'
valid_path = '/content/drive/MyDrive/fyp/fyp2/final models and datasets/dataset/dataset_new_with_self_label_final_5.0:3.0:3.0/valid_dataset.csv'

train_path = '/content/drive/MyDrive/fyp/fyp2/final models and datasets/dataset/dataset_new_with_self_label_3.0:4.0:4.0/train_dataset.csv'
valid_path ='/content/drive/MyDrive/fyp/fyp2/final models and datasets/dataset/dataset_new_with_self_label_3.0:4.0:4.0/valid_dataset.csv'

model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
#out = '/content/drive/MyDrive/fyp/fyp2/final models and datasets/sva_eval'
out = "/content/temp"

import shutil

if not os.path.exists(out):
  
  # Create a new directory because it does not exist 
  os.makedirs(out)
else:
  shutil.rmtree(out)
#shutil.rmtree(out)

name_of_text_column = "text"
#hyperparameter_tuning = True
reduce_ = False
num_hyper_trails = 3
hyper_size = 32
epoch = 12
train_type = "mixed"


df_train = pd.read_csv(train_path)

if train_type == "low":
  df_train = df_train[df_train['quality'] == "low"]
elif train_type == "high":
  df_train = df_train[df_train['quality'] == "high"]


df_valid = pd.read_csv(valid_path)

display(df_train)
display(df_valid)

,Unnamed: 0,text,label,source,quality
0,339,КриптоВести BTC 🤘 Blockasset Taps Well-Known ...,Positive,new_turk_low,low
1,747,Full thanks to as they have analysed this Bit...,Neutral,new_turk_high,high
2,301,"__crypto _tw 🔥 SanjiInu 🔥 💥 KYC, CHARITY, NF...",Neutral,self_manual,high
3,1743,$btc btc health check! No custom death cross; ...,Neutral,new_turk_low,low
4,1108,44 🏦FTX | BTC PERP 🦋 If you have trouble imag...,Neutral,new_turk_low,low
...,...,...,...,...,...
2785,683,This will be very useful in the future because...,Positive,new_turk_low,low
2786,1451,"And without Bitcoin Maxies hyping Bitcoin , th...",Positive,new_turk_low,low
2787,1106,11 IST 88.724 37996.605 ₿ 3371120.946 253...,Positive,new_turk_low,low
2788,1787,Bitcoin is Liberation Network.,Positive,new_turk_low,low


,Unnamed: 0,text,label,source,quality
0,71,Full thanks to as they have analysed this Bit...,Positive,incomplete_valid,high
1,266,Typing mistake for Satoshi not bitcoin,Neutral,incomplete_valid,high
2,410,Saitama SaitamaWolfPack SaitamaInu SaitaMask $...,Neutral,incomplete_valid,high
3,333,"100%, the nuclear industry has no idea what is...",Neutral,new_turk_high,high
4,401,airdrop Airdrops Crypto BSC DeFi Polygon Bitco...,Neutral,self_manual,high
...,...,...,...,...,...
1190,171,This is a very good project so don't miss to j...,Positive,new_turk_high,high
1191,319,Whoever didn’t panic sell we made it 🤝🎉 BTC,Positive,new_turk_high,high
1192,896,"To know more about IPOs and Unlisted shares, c...",Neutral,new_turk_high,high
1193,425,Soon to the moon 🌜 Unitycol $Unity Unityprot...,Positive,incomplete_valid,high


In [5]:
df_train['label'] = df_train['label'].apply(ConvertLabel2ModelLabel)
df_valid['label'] = df_valid['label'].apply(ConvertLabel2ModelLabel)

df_train = df_train[["text","label"]]
df_valid = df_valid[["text","label"]]
display(df_train)
display(df_valid)


,text,label
0,КриптоВести BTC 🤘 Blockasset Taps Well-Known ...,2
1,Full thanks to as they have analysed this Bit...,1
2,"__crypto _tw 🔥 SanjiInu 🔥 💥 KYC, CHARITY, NF...",1
3,$btc btc health check! No custom death cross; ...,1
4,44 🏦FTX | BTC PERP 🦋 If you have trouble imag...,1
...,...,...
2785,This will be very useful in the future because...,2
2786,"And without Bitcoin Maxies hyping Bitcoin , th...",2
2787,11 IST 88.724 37996.605 ₿ 3371120.946 253...,2
2788,Bitcoin is Liberation Network.,2


,text,label
0,Full thanks to as they have analysed this Bit...,2
1,Typing mistake for Satoshi not bitcoin,1
2,Saitama SaitamaWolfPack SaitamaInu SaitaMask $...,1
3,"100%, the nuclear industry has no idea what is...",1
4,airdrop Airdrops Crypto BSC DeFi Polygon Bitco...,1
...,...,...
1190,This is a very good project so don't miss to j...,2
1191,Whoever didn’t panic sell we made it 🤝🎉 BTC,2
1192,"To know more about IPOs and Unlisted shares, c...",1
1193,Soon to the moon 🌜 Unitycol $Unity Unityprot...,2


In [6]:
if reduce_:
  df_train = df_train[0:200]



dataset = ds.dataset(pa.Table.from_pandas(df_valid).to_batches())
### convert to Huggingface dataset
validation_dataset_torch = Dataset(pa.Table.from_pandas(df_valid))

dataset = ds.dataset(pa.Table.from_pandas(df_train).to_batches())
### convert to Huggingface dataset
training_dataset_torch = Dataset(pa.Table.from_pandas(df_train))


print(training_dataset_torch)
print(validation_dataset_torch)


Dataset({
    features: ['text', 'label'],
    num_rows: 2790
})
Dataset({
    features: ['text', 'label'],
    num_rows: 1195
})


In [7]:


def balanced_df(df_):
  #print(df.shape[0])
  df = df_.copy()
  df = df[['text','label']]

  df_pos = df[df['label'] == 2].sample(frac=1).reset_index(drop=True)
  df_neg = df[df['label'] == 0].sample(frac=1).reset_index(drop=True)
  df_neu = df[df['label'] == 1 ].sample(frac=1).reset_index(drop=True)
  

  pos = df_pos.shape[0]
  a = pos
  #print(a)
  neg = df_neg.shape[0]
  b = neg
  #print(b)
  neu = df_neu.shape[0]
  c = neu
  #print(c)
  #print(a+b+c)
  if a < b and a < c :
    smallest = a
  if b < a and b < c :
    smallest = b
  if c < a and c < b :
    smallest = c

  smallest = int(smallest/2)

  li_test = [df_pos[0:smallest],df_neg[0:smallest],df_neu[0:smallest]]
  li_valid = [df_pos[smallest:smallest*2],df_neg[smallest:smallest*2],df_neu[smallest:smallest*2]]
  #print("check")
  #display(df_pos[0:smallest])
  unused_li = [df_pos[smallest*2:],df_neg[smallest*2:],df_neu[smallest*2:]]
  

  return  pd.concat(li_valid,axis=0).sample(frac=1).reset_index(drop=True),pd.concat(li_test,axis=0).sample(frac=1).reset_index(drop=True), pd.concat(unused_li,axis=0).sample(frac=1).reset_index(drop=True)






In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name,model_max_length=512)



In [9]:
def tokenize_function(data):
    return tokenizer(data['text'], padding="max_length", truncation=True,)

print(type(training_dataset_torch))
train_dataset = training_dataset_torch.map(tokenize_function, batched=True)
eval_dataset = validation_dataset_torch.map(tokenize_function, batched=True)

print(train_dataset)
print(eval_dataset)




<class 'datasets.arrow_dataset.Dataset'>


  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 2790
})
Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1195
})


In [10]:
import numpy as np
from datasets import load_metric

metric = load_metric("matthews_correlation")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [11]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
#model.to(device)

In [12]:
from transformers import Trainer
from transformers import TrainingArguments
#learning_rate = 5e-05
batch_size = 8
#eval_batch_size = 4
seed = 40
#optimizer = Adam 
#with betas=(0.9,0.999) and epsilon=1e-08
adam_beta1 = 0.9
adam_beta2 =0.999
lr_scheduler_type = "linear"
num_epochs = 15
#args = TrainingArguments("test_trainer",report_to="wandb" ,logging_strategy = "epoch",evaluation_strategy="epoch",learning_rate = learning_rate,num_train_epochs = num_epochs,lr_scheduler_type =lr_scheduler_type, adam_beta1 = adam_beta1,adam_beta2 =adam_beta2  )

args = TrainingArguments(
    #'/content/drive/MyDrive/fyp/fyp2/model/model2-supervised/' + f"{model_name}-finetuned-",
    #'/content/' + f"{model_name}-finetuned-",
    out,
    #report_to="wandb",
    overwrite_output_dir = True,
    logging_dir = out,
    evaluation_strategy = "epoch",
    logging_strategy = "epoch",
    #save_strategy = "NO",
    learning_rate=4.5682e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay=0.01,
    metric_for_best_model='matthews_correlation',
    save_total_limit = 2,
    save_strategy = "epoch",
    load_best_model_at_end = True,
    seed = 40
    #push_to_hub=True,
)

args = TrainingArguments(
      #'/content/drive/MyDrive/fyp/fyp2/model/model2-supervised/' + f"{model_name}-finetuned-",
      #'/content/' + f"{model_name}-finetuned-",
      out,
      #report_to="wandb",
      overwrite_output_dir = True,
      logging_dir = out,
      evaluation_strategy = "epoch",
      logging_strategy = "epoch",
      #save_strategy = "NO",
      learning_rate=2.6510704963161386e-06,
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      num_train_epochs=epoch,
      weight_decay=0.01,
      metric_for_best_model='matthews_correlation',
      save_total_limit = 2,
      save_strategy = "epoch",
      load_best_model_at_end = True,
      seed = 18
      #push_to_hub=True,
  )







trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
# training
train_result = trainer.train() 
logs = trainer.state.log_history
# compute train results
metrics = train_result.metrics
max_train_samples = len(train_dataset)
metrics["train_samples"] = min(max_train_samples, len(train_dataset))

# save train results
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)

# compute evaluation results
metrics = trainer.evaluate()
max_val_samples = len(eval_dataset)
metrics["eval_samples"] = min(max_val_samples, len(eval_dataset))

# save evaluation results
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

import pickle
save_logs_dir = out
name_logs = "logs"

my_file = save_logs_dir + "/" + name_logs + ".p"
print("savint to " , my_file)

pickle.dump( logs, open( my_file, "wb" )    ) 

ld = pickle.load( open( my_file, "rb" ) )


print(logs)
print(logs[2])
print(type(logs))

print(ld)
print(type(ld))
print(logs == ld)

  #trainer.evaluate()



The following columns in the training set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2790
  Num Epochs = 12
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4188


Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.930500,0.769836,0.454717
2,0.846600,0.741414,0.440205
3,0.793100,0.766427,0.279496
4,0.759600,0.715582,0.523058
5,0.710900,0.747925,0.498481
6,0.682300,0.748851,0.457507
7,0.647800,0.785086,0.486181
8,0.625500,0.798319,0.392011
9,0.596200,0.797859,0.455014
10,0.585400,0.823031,0.488834


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1195
  Batch size = 8
Saving model checkpoint to /content/temp/checkpoint-349
Configuration saved in /content/temp/checkpoint-349/config.json
Model weights saved in /content/temp/checkpoint-349/pytorch_model.bin
tokenizer config file saved in /content/temp/checkpoint-349/tokenizer_config.json
Special tokens file saved in /content/temp/checkpoint-349/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num

***** train metrics *****
  epoch                    =       12.0
  total_flos               =  8204055GF
  train_loss               =     0.6919
  train_runtime            = 0:29:20.99
  train_samples            =       2790
  train_samples_per_second =     19.012
  train_steps_per_second   =      2.378


***** eval metrics *****
  epoch                     =       12.0
  eval_loss                 =     0.7156
  eval_matthews_correlation =     0.5231
  eval_runtime              = 0:00:14.35
  eval_samples              =       1195
  eval_samples_per_second   =     83.259
  eval_steps_per_second     =     10.451
savint to  /content/temp/logs.p
[{'loss': 0.9305, 'learning_rate': 2.43014795495646e-06, 'epoch': 1.0, 'step': 349}, {'eval_loss': 0.7698359489440918, 'eval_matthews_correlation': 0.4547170090756245, 'eval_runtime': 14.3223, 'eval_samples_per_second': 83.436, 'eval_steps_per_second': 10.473, 'epoch': 1.0, 'step': 349}, {'loss': 0.8466, 'learning_rate': 2.2092254135967823e-06, 'epoch': 2.0, 'step': 698}, {'eval_loss': 0.7414141893386841, 'eval_matthews_correlation': 0.4402048153140311, 'eval_runtime': 14.2978, 'eval_samples_per_second': 83.58, 'eval_steps_per_second': 10.491, 'epoch': 2.0, 'step': 698}, {'loss': 0.7931, 'learning_rate': 1.988302872237104e-06, 'epoch': 3.0, 'step':

In [13]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from transformers import pipeline

def huggingface_model_out(directory,tweets):

  tokenizer = AutoTokenizer.from_pretrained(directory)
  model = AutoModelForSequenceClassification.from_pretrained(directory)
  classifier = pipeline("sentiment-analysis", model = model, tokenizer = tokenizer)
  df = pd.DataFrame(columns=['label','conf'])
  label_list = []
  #conf = []
  score_ = []
  for tweet in tweets:
    res = classifier(tweet)[0]
    lab = res['label']
    #label.append(lab)
    #conf.append(res['score'])
    if lab == "Positive" or lab == "POS" or lab == "LABEL_2" or lab == "positive"  :
      label = "Positive"
      score = 1
    elif lab == "Negative" or lab == "NEG" or lab == "LABEL_0" or lab == "negative":
      label = "Negative"
      score = -1
    elif lab == "Neutral" or lab == "NEU" or lab == "LABEL_1" or lab == "neutral":
      label = "Neutral"
      score = 0
    else:
      print(directory," error model" , label)
      score = np.nan
    label_list.append(label)  
    score_.append(score)

  return label_list,score_


#model_name = "svalabs/twitter-xlm-roberta-bitcoin-sentiment"


#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForSequenceClassification.from_pretrained(model_name)
#classifier = pipeline("sentiment-analysis", model = model, tokenizer = tokenizer)
#res = classifier(df.tweet.values.tolist())

#label,conf,score = huggingface_model_out(model_name,data_file[name_of_tweet_column])
#result = huggingface_model_out(model_name,df['tweet'])
#display(score)
#display(label)

#display(result)


def hugging_face_sentiment_input(df,model_name,name=None,name_of_tweet_column="text"):
  if name == None:
    name = model_name
  label,score = huggingface_model_out(model_name,df[name_of_tweet_column])
  df[name + '_score'] = score
  df[name + '_sent'] = label
  #df[name + '_conf'] = conf
  return df


def find_metrics(df):
  col_names = []
  accu = []
  matthew =[]
  true_col = 'label'
  for name in df:
    if name.endswith("_sent"):
      #df_out[name + "_accuracy"] = accuracy_score[df[true],df[name]]
      col_names.append(name[:-5])

  df_out = pd.DataFrame(columns = col_names)

  for name in col_names:
    print(name)
    true = df[true_col].tolist()
    pred = df[name+"_sent"].tolist()
    #print(true)
    accu.append( accuracy_score(true,pred)) 
    matthew.append( matthews_corrcoef(true,pred) )
  #df_out.loc['accuracy'] = accu
  df_out.loc["MATTHEW'S CORRELATION COEFFICIENT (MCC)"] = matthew
  return df_out.T.sort_values(by=["MATTHEW'S CORRELATION COEFFICIENT (MCC)"])






test_path = "/content/drive/MyDrive/fyp/fyp2/final models and datasets/dataset/dataset_new_with_self_label_final_5.0:3.0:3.0/test_dataset.csv"
df_test = pd.read_csv(test_path)

df_test = hugging_face_sentiment_input(df_test,"/content/temp/checkpoint-3546")
df_metric = find_metrics(df_test)
display(df_metric)

ValueError: ignored

In [ ]:
#!kill -9 -1